In [ ]:
!pip install ktrain

Before appling BERT model, I have applied RNN - LSTM model with pre-trained glove word Embeddings where I got maximum accuracy of approx. 78.7% of accuracy after lot of text preprocessing.

# By Bert Model

In [3]:
#import ktrain
import ktrain
from ktrain import text

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Importing libraries

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

Dataset loading and cleaning

In [42]:
#load dataset
dataset_train = pd.read_csv(
    '/content/drive/My Drive/nlp-getting-started/dataset_train.csv')
dataset_test = pd.read_csv(
    '/content/drive/My Drive/nlp-getting-started/test.csv')

# dataset_cleaning

lemmatizer = WordNetLemmatizer()

text_train = list(dataset_train['text'])
keyword_train = list(dataset_train['keyword'])
location_train = list(dataset_train['location'])

for i in range(len(text_train)):

    reviews = text_train[i].split()
    for q in range(len(reviews)):
        if reviews[q][:4]=='http':
            reviews[q]=''


    if type(location_train[i]) != float:
        reviews.append(location_train[i])
    
    text_train[i] = ' '.join(reviews)


text_test = list(dataset_test['text'])
keyword_test = list(dataset_test['keyword'])
location_test = list(dataset_test['location'])

for i in range(len(text_test)):
    

    reviews_test = text_test[i].split()
    for k in range(len(reviews_test)):
        if reviews_test[k][:4] == 'http':
            reviews_test[k] = ''

    if type(location_test[i]) != float:
        reviews_test.append(location_test[i])

    
    text_test[i] = ' '.join(reviews_test)



In [43]:
text_train_x=np.array(text_train) 
text_train_y=np.array(list(dataset_train['target']))

In [44]:
text_train[:30]

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask. Canada',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO

In [45]:
type(np.array(list(dataset_train['target']))[0])

numpy.int64

Load pre-trained BERT Model 

In [46]:
(x_train,y_train),(x_test,y_test),preprocess=text.texts_from_array(x_train=text_train_x,y_train=text_train_y,maxlen=76,preprocess_mode='bert',class_names=['0','1'],max_features=30000)

task: text classification
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [48]:
model=text.text_classifier('bert',train_data=(x_train,y_train),preproc=preprocess)

Is Multi-Label? False
maxlen is 76
done.


In [49]:
learner=ktrain.get_learner(model=model,train_data=(x_train,y_train),val_data=(x_test,y_test),batch_size=16)

Find Appropiate learning rate for BERT model

In [ ]:
learner.lr_find()
learner.lr_plot()

In [50]:
learner.autofit(lr=1e-5)

early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 1e-05...
Train on 6851 samples, validate on 762 samples
Epoch 1/1024
6851/6851 [==============================] - 328s 48ms/sample - loss: 0.4784 - accuracy: 0.7818 - val_loss: 0.3748 - val_accuracy: 0.8412
Epoch 2/1024
6851/6851 [==============================] - 306s 45ms/sample - loss: 0.3579 - accuracy: 0.8542 - val_loss: 0.3645 - val_accuracy: 0.8504
Epoch 3/1024
6851/6851 [==============================] - 305s 44ms/sample - loss: 0.2965 - accuracy: 0.8845 - val_loss: 0.3940 - val_accuracy: 0.8386
Epoch 4/1024
6848/6851 [============================>.] - ETA: 0s - loss: 0.2447 - accuracy: 0.9076
Epoch 00004: Reducing Max LR on Plateau: new max lr will be 5e-06 (if not early_stopping).
6851/6851 [==============================] - 305s 45ms/sample - loss: 0.2450 - accuracy: 0.9075 - val_loss: 0.4274 - val_accu

In [51]:
predictor=ktrain.get_predictor(learner.model,preproc=preprocess)

Final Predictions 

In [53]:
dataset_test['target']=predictor.predict(text_test)

In [54]:
y_pred_target=pd.concat([dataset_test['id'],dataset_test['target'].astype('int32')],axis=1)

In [55]:
y_pred_target

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


Save Final predictions

In [56]:
y_pred_target.to_csv('bert_final_3',index=False)

# LSTM Model

Firstly I applied LSTM model with pre-trained glove Word Embedding but got no better results

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max([len(l) for l in text_train])

176

In [ ]:
max_length=max([len(l) for l in text_train])

# embedding_dim=100
tokenizer=Tokenizer(oov_token='<OOV>')

tokenizer.fit_on_texts(text_train)
word_index=tokenizer.word_index
sequences = tokenizer.texts_to_sequences(text_train)
padded_train = pad_sequences(sequences, maxlen=70, padding='post', truncating='post')

sequences_pred= tokenizer.texts_to_sequences(text_test)
padded_test=pad_sequences(sequences_pred, maxlen=70, padding='post', truncating='post')


In [ ]:
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

In [ ]:
import tensorflow as tf

In [ ]:
embedding_dim=100

In [ ]:
embeddings_index = {}
vocab_size=len(word_index)
with open('/content/drive/My Drive/image captioning/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=70, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.Bidirectional(LSTM(256)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 40
training_padded=np.array(padded_train)
training_labels=np.array(dataset_train['target'],dtype='float')
history = model.fit(training_padded, training_labels, epochs=num_epochs,validation_split=0.05)

Epoch 1/40
226/226 [==============================] - 2s 9ms/step - loss: 0.5082 - accuracy: 0.7526 - val_loss: 0.4568 - val_accuracy: 0.8031
Epoch 2/40
226/226 [==============================] - 1s 6ms/step - loss: 0.4421 - accuracy: 0.8005 - val_loss: 0.4884 - val_accuracy: 0.7848
Epoch 3/40
226/226 [==============================] - 1s 6ms/step - loss: 0.4024 - accuracy: 0.8204 - val_loss: 0.4733 - val_accuracy: 0.7927
Epoch 4/40
226/226 [==============================] - 1s 6ms/step - loss: 0.3739 - accuracy: 0.8367 - val_loss: 0.4406 - val_accuracy: 0.8005
Epoch 5/40
226/226 [==============================] - 1s 6ms/step - loss: 0.3255 - accuracy: 0.8650 - val_loss: 0.4781 - val_accuracy: 0.7848
Epoch 6/40
226/226 [==============================] - 1s 6ms/step - loss: 0.2912 - accuracy: 0.8865 - val_loss: 0.5008 - val_accuracy: 0.8005
Epoch 7/40
226/226 [==============================] - 1s 6ms/step - loss: 0.2573 - accuracy: 0.8988 - val_loss: 0.5288 - val_accuracy: 0.7795
Epoch 

In [ ]:
testing_data=np.array(padded_test)
result=model.predict(testing_data)
result[result>=0.5]=1
result[result<0.5]=0

In [ ]:
result.shape

(3263, 1)

In [ ]:
result[:20] 

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
result=result.reshape((result.shape[0],))

In [ ]:
y_pred=pd.concat([dataset_test,pd.Series(result,name='target').astype('int32')],axis=1)

In [ ]:
value_count_keyword=dict(dataset_train['keyword'].value_counts())

In [ ]:
dataset_train[dataset_train['keyword']=='ruin']['keyword'].value_counts()

ruin    37
Name: keyword, dtype: int64

In [ ]:
value_count_keyword_zero=dict(dataset_train[dataset_train['target']==0]['keyword'].value_counts())

In [ ]:
value_count_keyword_one=dict(dataset_train[dataset_train['target']==1]['keyword'].value_counts())

In [ ]:
for i in list(value_count_keyword.keys()):
  try:
    if (int(value_count_keyword[i]) >= 35) and ((int(value_count_keyword[i]) - int(value_count_keyword_zero[i])) < 4):
      y_pred.loc[y_pred['keyword'] == i,'target']=0
  except KeyError:
    pass

In [ ]:
for i in list(value_count_keyword.keys()):
  if (int(value_count_keyword[i]) >= 35) and ((int(value_count_keyword[i]) - int(value_count_keyword_one[i])) < 4):
    y_pred.loc[y_pred['keyword'] == i,'target']=1


In [ ]:
y_pred_target=pd.concat([dataset_test['id'],y_pred['target'].astype('int32')],axis=1)

In [ ]:
y_pred_target

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,0
3261,10874,0


In [ ]:
y_pred_target.to_csv('cnn_rnn_final',index=False)